In [19]:
import pyhdb
import yaml
import pandas as pd

In [6]:
with open('../credentials.yml', 'r') as f:
    credentials = yaml.load(f)

In [10]:
connection = pyhdb.connect(**credentials)
cursor = connection.cursor()

In [11]:
# Distinct years contain 0
query = 'SELECT DISTINCT VISITYEAR FROM TUKGRP4."Transcript"'
cursor.execute(query)
cursor.fetchall()

[(0,), (2009,), (2010,), (2011,), (2012,)]

In [18]:
# Select doc visits by age group
query = \
'''
    SELECT agegroup, AVG(visits), MIN(visits), MAX(visits), MEDIAN(visits)
    FROM (
    SELECT *,
      CASE
      WHEN AGE < 10 THEN 1
      WHEN AGE < 20 THEN 2
      WHEN AGE < 30 THEN 3
      WHEN AGE < 40 THEN 4
      WHEN AGE < 50 THEN 5
      WHEN AGE < 60 THEN 6
      WHEN AGE < 70 THEN 7
      WHEN AGE < 80 THEN 8
      WHEN AGE < 90 THEN 9
      ELSE 10 END AS agegroup
    FROM (
      SELECT
        t.PATIENTGUID,
        2012 - p.YEAROFBIRTH as age,
        COUNT(t.VISITYEAR) as visits
      FROM TUKGRP4."Transcript" AS t
        JOIN TUKGRP4."Patient" AS p ON t.PATIENTGUID = p.PATIENTGUID
      WHERE t.VISITYEAR != 0
      GROUP BY t.PATIENTGUID, p.YEAROFBIRTH
    )
    ) GROUP BY agegroup;
'''
cursor.execute(query)
agg_data = cursor.fetchall()

In [24]:
df = pd.DataFrame(agg_data)
df.columns = ['group', 'avg', 'min', 'max', 'median']
df

,group,avg,min,max,median
0,10,14.871794,3,41,11
1,9,15.833333,1,136,12
2,8,13.351435,1,130,10
3,7,13.399787,3,275,9
4,6,13.010355,1,260,9
5,5,11.945614,1,252,8
6,4,11.146517,1,218,7
7,3,11.148331,1,104,8
8,2,10.163461,2,92,7
